In [ ]:
    

''' 
In absence of GPU (only CPU): 
pip install torch torchvision torchaudio
pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
need to install visual studio with right depedencies:
On the Workloads tab, check the option "Desktop development with C++".

On the Individual Components tab, make sure the following options are checked:

MSVC v142 - VS 2019 C++ x64/x86 build tools

Windows 10 SDK

C++ x64/x86 build tools

Update:
was necessary to reinstall and install as follows:

pip uninstall torch torch-geometric torch-geometric-temporal torch-scatter torch-sparse torch-cluster torch-spline-conv

pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
pip install torch-geometric-temporal


'''
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import networkx as nx

from torch_geometric_temporal.nn.recurrent import TGN
from torch_geometric_temporal.signal import temporal_signal_split
from torch_geometric_temporal.dataset import JODIEDataset

# Carregar dataset JODIE (substituir por um dataset real de transações)
dataset = JODIEDataset(name='wikipedia')  # Placeholder, pode ser trocado por dados reais
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

# Definir o modelo TGN
class FraudDetector(torch.nn.Module):
    def __init__(self, node_features, memory_dim=128, time_dim=128):
        super(FraudDetector, self).__init__()
        self.tgn = TGN(node_features, memory_dim, time_dim)
        self.fc = torch.nn.Linear(memory_dim, 1)  # Camada final para classificação binária

    def forward(self, x, edge_index, edge_attr, time, memory):
        memory, embeddings = self.tgn(x, edge_index, edge_attr, time, memory)
        return torch.sigmoid(self.fc(embeddings)), memory

# Inicializar modelo
node_features = dataset.num_node_features
model = FraudDetector(node_features)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCELoss()
loss_history = []

# Treinamento
for epoch in range(10):  # Número de épocas ajustável
    model.train()
    memory = None
    loss_epoch = 0
    for snapshot in train_dataset:
        optimizer.zero_grad()
        y_pred, memory = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr, snapshot.t, memory)
        loss = criterion(y_pred.squeeze(), snapshot.y.float())
        loss.backward()
        optimizer.step()
        loss_epoch += loss.item()
    loss_history.append(loss_epoch)
    print(f'Epoch {epoch + 1}, Loss: {loss_epoch:.4f}')

# Visualizar a curva de perda
plt.plot(range(1, 11), loss_history, marker='o')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.title('Evolução da Perda Durante o Treinamento')
plt.show()

# Avaliação
model.eval()
memory = None
correct = 0
total = 0
with torch.no_grad():
    for snapshot in test_dataset:
        y_pred, memory = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr, snapshot.t, memory)
        predicted = (y_pred.squeeze() > 0.5).float()
        correct += (predicted == snapshot.y.float()).sum().item()
        total += snapshot.y.shape[0]

accuracy = correct / total
print(f'Accuracy: {accuracy:.4f}')

# Visualizar o grafo
snapshot = next(iter(test_dataset))
G = nx.Graph()
for i, (src, dst) in enumerate(snapshot.edge_index.t().tolist()):
    G.add_edge(src, dst)
nx.draw(G, with_labels=True, node_color='lightblue', edge_color='gray')
plt.title('Visualização da Rede de Transações')
plt.show()


OSError: [WinError 127] The specified procedure could not be found

In [2]:
import torch
print(torch.cuda.is_available())

False


In [2]:
import torch
from torch_geometric_temporal.nn.recurrent import TGN

# Test if TGN can be initialized
node_features = 10
memory_dim = 128
time_dim = 128
tgn = TGN(node_features, memory_dim, time_dim)
print("TGN initialized successfully!")

OSError: [WinError 127] The specified procedure could not be found

In [4]:
import torch_geometric